In [831]:
import pandas as pd
import numpy as np
import random
import dask # for faster import
import dask.dataframe as dd 
import dask.array as da
from datetime import datetime
import gc
from tqdm import tqdm

In [832]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Loading the full datasets

In [833]:
path = '../../'

col_to_use = ['STORE_ID', 'DATE', 'INDIV_ID',
       'VEHICLE_ID', 'UNITS', 'SALES', 'STATE_CODE', 'ZIP_CODE', 'MAKE',
       'MODEL', 'SUB_MODEL', 'MODEL_YEAR', 'PROD_GROUP_CODE',
       'PROD_GROUP_DESC', 'CATEGORY_DESC',
       'SEGMENT_DESC', 'CLASS_DESC', 'DISCOUNT_FLAG',
       'CROSS_SECTION', 'ASPECT_RATIO', 'RIM_SIZE']

faster import for a large dataset

In [834]:
dy = pd.DataFrame()

#we are going to work with chunks of size 1 million rows
chunksize = 10 ** 6

for chunk in tqdm(pd.read_csv(path + 'all_years.csv', chunksize=chunksize)):
    filtered = (chunk[(np.where(chunk['DISCOUNT_FLAG']!='Y', True, False))])
    dy = pd.concat([dy, filtered], ignore_index=True)

In [835]:
# the random sample is skipping 999 rows and taking 1 row on average 
# dy15 = pd.read_csv(path + 'AprOct15_trimmed.csv', usecols = col_to_use)#, skiprows = lambda i: i>0 and random.random() > 0.1)

In [836]:
# dy16 = pd.read_csv(path + 'JanOct16_trimmed.csv', usecols = col_to_use) 

In [837]:
# dy17 = pd.read_csv(path + 'JanOct17_trimmed.csv', usecols = col_to_use)

In [838]:
# dy18 = pd.read_csv(path + 'JanOct18_trimmed.csv', usecols = col_to_use)

In [839]:
dn = pd.read_csv(path + 'Nov17_trimmed.csv', usecols = col_to_use)

In [840]:
# dy = pd.concat([dy15, dy16, dy17])

In [841]:
# dy.to_csv(path + 'all_years.csv')

In [842]:
# ## using dask instead of pandas to fasten dataset import
# dtype={'ASPECT_RATIO': 'object',
#        'CROSS_SECTION': 'object',
#        'RIM_SIZE': 'object'}

In [843]:
# dy = dd.read_csv(path + 'all_years.csv', dtype = dtype)
# dy = pd.read_csv(path + 'all_years.csv', skiprows = lambda i: i>0 and random.random() > 0.0001)

# dy = pd.read_csv(path + 'all_years.csv', nrows = 1000)

In [844]:
dy.head()

,Unnamed: 0,STORE_ID,DATE,INDIV_ID,VEHICLE_ID,UNITS,SALES,STATE_CODE,ZIP_CODE,MSA,MAKE,MODEL,SUB_MODEL,MODEL_YEAR,PROD_GROUP_CODE,PROD_GROUP_DESC,CATEGORY_DESC,SEGMENT_DESC,CLASS_DESC,DISCOUNT_FLAG,CROSS_SECTION,ASPECT_RATIO,RIM_SIZE
0,0,26484,2015-04-06,295141827.0,935107107,1.0,6.99,IL,60169,"CHICAGO,IL",TOYOTA,COROLLA LE,NaN,2012.0,4.0,Services,Tire Services,Tire Services,Tire Pressure Monitoring Systems,N,NaN,NaN,NaN
1,1,26484,2015-04-06,295141827.0,935107107,0.0,0.00,IL,60169,"CHICAGO,IL",TOYOTA,COROLLA LE,NaN,2012.0,4.0,Services,Miscellaneous Services,Inspection Services,Courtesy Check,N,NaN,NaN,NaN
2,2,26484,2015-04-06,295141827.0,935107107,0.0,1.99,IL,60169,"CHICAGO,IL",TOYOTA,COROLLA LE,NaN,2012.0,4.0,Services,Tire Services,Tire Services,Misc. Tire Service,N,NaN,NaN,NaN
3,3,26484,2015-04-06,295141827.0,935107107,0.0,10.00,IL,60169,"CHICAGO,IL",TOYOTA,COROLLA LE,NaN,2012.0,4.0,Services,Tire Services,Tire Services,Misc. Tire Service,N,NaN,NaN,NaN
4,4,26484,2015-04-06,295141827.0,935107107,0.0,2.00,IL,60169,"CHICAGO,IL",TOYOTA,COROLLA LE,NaN,2012.0,4.0,Services,Tire Services,Tire Services,Tire Pressure Monitoring Systems,N,NaN,NaN,NaN


In [845]:
dy.shape

(109858983, 23)

## setting the new dataframe

In [846]:
vehicles = dy.drop_duplicates(subset = 'VEHICLE_ID',keep='first')

In [847]:
targets = dn[dn['PROD_GROUP_CODE'] == 5]['INDIV_ID'].unique()

In [848]:
unique_IDs = dy.INDIV_ID.unique()

In [849]:
year_indivs = pd.DataFrame(unique_IDs, columns = ['indiv'])

In [850]:
year_indivs.head()

,indiv
0,295141827.0
1,303947384.0
2,486895284.0
3,303949519.0
4,301787904.0


In [851]:
year_indivs['label'] = np.where(year_indivs['indiv'].isin(targets), 1, 0)

In [852]:
dy.shape

(109858983, 23)

In [853]:
year_indivs['cust_in_top_10_sales'] = np.where(year_indivs['indiv'].isin(dy.INDIV_ID.value_counts().nlargest(int(len(year_indivs)/10))),1,0)

In [854]:
year_indivs.label.value_counts()

0    4765799
1     111421
Name: label, dtype: int64

In [855]:
len(year_indivs)

4877220

### Feature : is the store among the top performers regarding total number of transactions in general?

In [856]:
top_stores_trans_list = dy.STORE_ID.value_counts().nlargest(int(len(dy)/100)).index

In [857]:
len(top_stores_trans_list)

2275

get individuals visiting those stores

In [858]:
indiv_store_list = dy[dy['STORE_ID'].isin(top_stores_trans_list)]['INDIV_ID']

In [859]:
year_indivs['store_top_tran'] = np.where(year_indivs.indiv.isin(indiv_store_list), 1, 0)

In [860]:
# sanity check
year_indivs['store_top_tran'].value_counts()

1    4877220
Name: store_top_tran, dtype: int64

In [861]:
del top_stores_trans_list
del indiv_store_list

gc.collect()

160

### Feature : is the store among the top performers regarding total number of TIRES transactions ?

In [862]:
tires = dy.query('PROD_GROUP_CODE == 5')

In [863]:
top_stores_tires_list = tires.STORE_ID.value_counts().nlargest(int(len(dy)/100)).index

In [864]:
top_stores_tires_list

Int64Index([237119, 240187, 326506, 517895, 517585, 323022, 517658, 326507,
            517909, 303599,
            ...
             18287, 122106, 787729, 781999, 248065, 121916, 227596, 787742,
            787741, 324008],
           dtype='int64', length=2275)

get individuals visiting those stores

In [865]:
indiv_store_tires_list = dy[dy['STORE_ID'].isin(top_stores_tires_list)]['INDIV_ID']

In [866]:
year_indivs['store_top_tires_tran'] = np.where(year_indivs.indiv.isin(indiv_store_tires_list), 1, 0)

In [867]:
# sanity check
year_indivs['store_top_tires_tran'].value_counts()

1    4877220
Name: store_top_tires_tran, dtype: int64

In [868]:
year_indivs.head()

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran
0,295141827.0,0,0,1,1
1,303947384.0,0,0,1,1
2,486895284.0,0,0,1,1
3,303949519.0,0,0,1,1
4,301787904.0,0,0,1,1


In [869]:
del indiv_store_tires_list
del top_stores_tires_list

gc.collect()

20

## Feature: has this vehicle recently had a tire service? 

consider adding a certain time limit 

### list of vehicles with recent tires' services 

Here I will remove VEHICLE_ID = 1   These cars don't have MAKE, MODEL, MODEL YEAR. it seems that data is missing regarding the vehicle description

In [870]:
vhc_tire_svc_array = dy.query('CATEGORY_DESC == "Tire Services" & VEHICLE_ID != 1')['VEHICLE_ID'].unique()

In [871]:
indiv_vhc_svc_list = dy[dy['VEHICLE_ID'].isin(vhc_tire_svc_array)]['INDIV_ID']

In [872]:
year_indivs['vehicle_tire_svc'] = np.where(year_indivs.indiv.isin(indiv_vhc_svc_list), 1, 0)

In [873]:
# sanity check
year_indivs['vehicle_tire_svc'].value_counts()

1    3495896
0    1381324
Name: vehicle_tire_svc, dtype: int64

### How many transactions did these individuals [who came at least once for tire services] have with bridgestone before?

In [874]:
number_trans_for_ind_with_tire_services = indiv_vhc_svc_list.value_counts().reset_index()

number_trans_for_ind_with_tire_services.columns = ['indiv', 'number_trans']
# year_indivs['number_trans_for_ind_with_tire_services'] = np.where(year_indivs.indiv.isin(number_trans_for_ind_with_tire_services.index),
#                                                                   number_trans_for_ind_with_tire_services.INDIV_ID, 0)

number_trans_for_ind_with_tire_services.set_index("indiv")
# number_trans_for_ind_with_tire_services.head()
year_indivs = year_indivs.merge(number_trans_for_ind_with_tire_services, left_on = 'indiv', right_on = 'indiv', how = 'left')


In [875]:
year_indivs.number_trans = year_indivs.number_trans.fillna(0)

In [876]:
year_indivs.head()

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans
0,295141827.0,0,0,1,1,1,13.0
1,303947384.0,0,0,1,1,0,0.0
2,486895284.0,0,0,1,1,1,103.0
3,303949519.0,0,0,1,1,1,10.0
4,301787904.0,0,0,1,1,1,9.0


In [877]:
year_indivs.shape

(4877220, 7)

### Can we add only transaction if they occured different days? This may mean the number of visits

In [878]:
indiv_vhc_svc_with_dates = dy[dy['VEHICLE_ID'].isin(vhc_tire_svc_array)][['INDIV_ID', 'DATE']]

In [879]:
indiv_vhc_svc_with_dates.head()

,INDIV_ID,DATE
0,295141827.0,2015-04-06
1,295141827.0,2015-04-06
2,295141827.0,2015-04-06
3,295141827.0,2015-04-06
4,295141827.0,2015-04-06


In [880]:
number_visits_for_ind_with_tire_services = indiv_vhc_svc_with_dates.groupby('INDIV_ID').agg({'DATE': 'nunique'}).reset_index()

number_visits_for_ind_with_tire_services.columns = ['indiv', 'number_visits'] 

year_indivs = year_indivs.merge(number_visits_for_ind_with_tire_services, left_on = 'indiv', right_on = 'indiv', how = 'left')

In [881]:
year_indivs.head(100)

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans,number_visits
0,295141827.0,0,0,1,1,1,13.0,2.0
1,303947384.0,0,0,1,1,0,0.0,NaN
2,486895284.0,0,0,1,1,1,103.0,15.0
3,303949519.0,0,0,1,1,1,10.0,1.0
4,301787904.0,0,0,1,1,1,9.0,1.0
5,536567267.0,0,0,1,1,1,8.0,2.0
6,254367352.0,0,0,1,1,1,10.0,2.0
7,259258499.0,0,0,1,1,1,23.0,3.0
8,300257974.0,0,0,1,1,1,12.0,1.0
9,304508907.0,0,0,1,1,1,6.0,1.0


In [882]:
year_indivs.number_visits = year_indivs.number_visits.fillna(0)

In [883]:
# sanity check
year_indivs.number_visits.equals(year_indivs.number_trans)

False

In [884]:
year_indivs.shape

(4877220, 8)

## Feature: how many times did the individual come to purchase tires in the past? I will consider trasnactions within the same day as one visit

In [885]:
tire_pur_indiv = dy.query('PROD_GROUP_DESC == "Tires" & SEGMENT_DESC != "Others"')[['INDIV_ID', 'DATE']]

In [886]:
tire_pur_indiv.head()

,INDIV_ID,DATE
24,303949519.0,2015-04-15
64,300257974.0,2015-04-22
65,300257974.0,2015-04-22
71,304508907.0,2015-04-25
90,304654370.0,2015-04-28


In [887]:
number_visits_tire_purch = tire_pur_indiv.groupby('INDIV_ID').agg({'DATE': 'nunique'}).reset_index()

number_visits_tire_purch.columns = ['indiv', 'number_visits_tire_purchase'] 

year_indivs = year_indivs.merge(number_visits_tire_purch, left_on = 'indiv', right_on = 'indiv', how = 'left')

In [888]:
year_indivs.number_visits_tire_purchase = year_indivs.number_visits_tire_purchase.fillna(0)

In [889]:
year_indivs.head()

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans,number_visits,number_visits_tire_purchase
0,295141827.0,0,0,1,1,1,13.0,2.0,0.0
1,303947384.0,0,0,1,1,0,0.0,0.0,0.0
2,486895284.0,0,0,1,1,1,103.0,15.0,2.0
3,303949519.0,0,0,1,1,1,10.0,1.0,1.0
4,301787904.0,0,0,1,1,1,9.0,1.0,0.0


In [890]:
year_indivs.shape

(4877220, 9)

## Feature: has anyone purchased a tire for this vehicle before?

we need to settle on the definition of tire purchase, is it 5 only or after removing others as well?

In [891]:
vhc_tire_pur_array = dy.query('PROD_GROUP_DESC == "Tires" & SEGMENT_DESC != "Others" & VEHICLE_ID != 1')['VEHICLE_ID'].unique()

In [892]:
vhc_tire_pur_array

array([938363657, 938571675, 938630657, ..., 962509271, 963418995,
       920439678])

In [893]:
indiv_vhc_pur_list = dy[dy['VEHICLE_ID'].isin(vhc_tire_pur_array)]['INDIV_ID']

In [894]:
year_indivs['vhc_tire_purchase'] = np.where(year_indivs.indiv.isin(indiv_vhc_pur_list), 1, 0)

In [895]:
# sanity check
year_indivs['vhc_tire_purchase'].value_counts()

0    2507324
1    2369896
Name: vhc_tire_purchase, dtype: int64

## Feature : when was the last time this vehicle bought a new tire? [last year - 2 years - 3 years]

In [896]:
dy['DATE'] = pd.to_datetime(dy['DATE'])

In [897]:
dy['year'] = pd.DatetimeIndex(dy['DATE']).year
dy['month'] = pd.DatetimeIndex(dy['DATE']).month

In [898]:
dy_vhc_tire_purch = dy.query('PROD_GROUP_DESC == "Tires" & SEGMENT_DESC != "Others"')

In [899]:
month_look = dy_vhc_tire_purch.groupby('VEHICLE_ID').agg({'month':['max'] })

Max is the latest month the tire was purchased. If the last month to purchase a tire was before 6, the vehicle may need to replace tires. 
Of course, this needs a change, but after we combine datasets of different years. 

In [900]:
month_look.columns = ['last_mo_tire_purch']

In [901]:
vhc_early_purch_tires = month_look.reset_index().query('last_mo_tire_purch < 6')['VEHICLE_ID']

In [902]:
indiv_vhc_early_pur_list = dy[dy['VEHICLE_ID'].isin(vhc_early_purch_tires)]['INDIV_ID']

In [903]:
year_indivs['vhc_early_tire_purchase'] = np.where(year_indivs.indiv.isin(indiv_vhc_early_pur_list), 1, 0)

In [904]:
# sanity check
year_indivs['vhc_early_tire_purchase'].value_counts()

0    3843910
1    1033310
Name: vhc_early_tire_purchase, dtype: int64

## Feature : when was the last time this vehicle bought a new tire? (numerical) 
Reference point is Oct 17 for training and Oct 18 for testing. 
This will be automatic by getting max date

For example, the value should be 3 if purchased 3 months ago. 

In [905]:
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

In [906]:
d1 = dy.DATE.max()

In [907]:
last_purch_per_indiv = dy_vhc_tire_purch.groupby('INDIV_ID').agg({'DATE':'max'}).reset_index()

last_purch_per_indiv.columns = ['indiv', 'last_tire_purchase'] 

year_indivs = year_indivs.merge(last_purch_per_indiv, left_on = 'indiv', right_on = 'indiv', how = 'left')


year_indivs['diff_from_last_tire_purch'] = diff_month(d1, pd.DatetimeIndex(year_indivs.last_tire_purchase))

year_indivs.head()

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans,number_visits,number_visits_tire_purchase,vhc_tire_purchase,vhc_early_tire_purchase,last_tire_purchase,diff_from_last_tire_purch
0,295141827.0,0,0,1,1,1,13.0,2.0,0.0,0,0,NaT,NaN
1,303947384.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN
2,486895284.0,0,0,1,1,1,103.0,15.0,2.0,1,1,2017-10-14,0.0
3,303949519.0,0,0,1,1,1,10.0,1.0,1.0,1,1,2015-04-15,30.0
4,301787904.0,0,0,1,1,1,9.0,1.0,0.0,0,0,NaT,NaN


### Here I need to understand what to do with units 

In [908]:
dy_vhc_tire_purch.head()  # dataset created before for tires purchased 

,Unnamed: 0,STORE_ID,DATE,INDIV_ID,VEHICLE_ID,UNITS,SALES,STATE_CODE,ZIP_CODE,MSA,MAKE,MODEL,SUB_MODEL,MODEL_YEAR,PROD_GROUP_CODE,PROD_GROUP_DESC,CATEGORY_DESC,SEGMENT_DESC,CLASS_DESC,DISCOUNT_FLAG,CROSS_SECTION,ASPECT_RATIO,RIM_SIZE,year,month
24,38,244183,2015-04-15,303949519.0,938363657,4.0,343.96,IA,52806,NaN,DODGE,JOURNEY R/T,R/T,2013.0,5.0,Tires,Passenger Tires,Touring,Mass Market Tires,N,225,65,17,2015,4
64,108,591467,2015-04-22,300257974.0,938571675,0.0,33.04,MO,63128,"ST.LOUIS,M",FORD,FOCUS SE,SE,2008.0,5.0,Tires,Road Hazard,Road Hazard,Road Hazard,N,NONE,NONE,NONE,2015,4
65,109,591467,2015-04-22,300257974.0,938571675,4.0,227.96,MO,63128,"ST.LOUIS,M",FORD,FOCUS SE,SE,2008.0,5.0,Tires,Passenger Tires,Touring,Economy Tires,N,195,60,15,2015,4
71,117,19755,2015-04-25,304508907.0,938630657,4.0,416.96,NJ,7002,JERSEY CIT,FORD,EXPLORER XLT,XLT,2002.0,5.0,Tires,Passenger Tires,P-Metric CUV/SUV,CUV/SUV Highway All-Season,N,235,70,16,2015,4
90,152,278633,2015-04-28,304654370.0,938742156,2.0,163.98,PA,19031,NaN,CHEVROLET,LUMINA BASE,Base,1999.0,5.0,Tires,Passenger Tires,Touring,Mass Market Tires,N,225,60,16,2015,4


In [909]:
dy.columns

Index(['Unnamed: 0', 'STORE_ID', 'DATE', 'INDIV_ID', 'VEHICLE_ID', 'UNITS',
       'SALES', 'STATE_CODE', 'ZIP_CODE', 'MSA', 'MAKE', 'MODEL', 'SUB_MODEL',
       'MODEL_YEAR', 'PROD_GROUP_CODE', 'PROD_GROUP_DESC', 'CATEGORY_DESC',
       'SEGMENT_DESC', 'CLASS_DESC', 'DISCOUNT_FLAG', 'CROSS_SECTION',
       'ASPECT_RATIO', 'RIM_SIZE', 'year', 'month'],
      dtype='object')

In [910]:
dy_vhc_tire_purch.groupby(['CATEGORY_DESC', 'SEGMENT_DESC', 'CLASS_DESC'], dropna = False).agg({'UNITS': ['min', 'nunique']})

UNITS  \
                                                                                       min   
CATEGORY_DESC     SEGMENT_DESC                     CLASS_DESC                                
Light Truck Tires Commercial Tires                 All Terrain Tires                  -6.0   
                                                   Highway Tires                      -6.0   
                  Commercial/Recreational          All-Terrain                         0.0   
                                                   Highway All-Season                  0.0   
                                                   Maximum Traction                    0.0   
                  Recreational Tires               All Terrain Tires                  -7.0   
                                                   Highway Tires                      -6.0   
                                                   Max Traction Tires                 -5.0   
                  Snow Tires-LT                    Snow Tires - All Terrain LT         1.0   
                                                   Snow Tires - Commercial LT          1.0   
Others            Agricultural Tires               Garden Tires                        0.0   
                                                   Implement Tires/Industrial          0.0   
                                                   Rear Ag Tires                       2.0   
                  Industrial Tires                 Industrial Tires                    0.0   
                  Medium Truck Tires               Radial Tires                        0.0   
                  Miscellaneous Tire Related Items Misc/Discounts/Non-Inventoreable  -12.0   
                                                   Other                              -4.0   
                  Tubes                            Tubes                              -5.0   
Passenger Tires   Other Passenger Tires            Other Passenger Tires             -20.0   
                  P Metric Light Truck Tires       All Terrain Tires                  -5.0   
                                                   Highway Tires                      -8.0   
                  P-Metric CUV/SUV                 CUV/SUV All-Terrain                 1.0   
                                                   CUV/SUV Highway All-Season         -8.0   
                                                   CUV/SUV Sport Performance Summer    0.0   
                  Performance Tires                Performance All-Season             -4.0   
                                                   S/T Performance Tires              -4.0   
                                                   Touring H/V/Z Tires                -6.0   
                                                   Ultra High Performance All Season  -4.0   
                                                   Ultra High Performance Summer      -4.0   
                  Snow Tires-PS                    Snow Tires                         -4.0   
                  Touring                          Car & Minivan All-Season           -4.0   
                                                   Economy Tires                     -10.0   
                                                   Mass Market Tires                  -5.0   
Road Hazard       Road Hazard                      Road Hazard                         0.0   

                                                                                              
                                                                                     nunique  
CATEGORY_DESC     SEGMENT_DESC                     CLASS_DESC                                 
Light Truck Tires Commercial Tires                 All Terrain Tires                      16  
                                                   Highway Tires                          23  
                  Commercial/Recreational          All-Terrain                             7  
                                    

It seems that zeros in the units do actually make sense. 

## For those who purchased tires, how many units did they purchase for all years?

In [911]:
units_tires_purch = dy_vhc_tire_purch.groupby('INDIV_ID').agg({'UNITS':'sum'}).reset_index()

units_tires_purch.columns = ['indiv', 'units_tires_purch'] 

year_indivs = year_indivs.merge(units_tires_purch, left_on = 'indiv', right_on = 'indiv', how = 'left')

year_indivs.head()

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans,number_visits,number_visits_tire_purchase,vhc_tire_purchase,vhc_early_tire_purchase,last_tire_purchase,diff_from_last_tire_purch,units_tires_purch
0,295141827.0,0,0,1,1,1,13.0,2.0,0.0,0,0,NaT,NaN,NaN
1,303947384.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN
2,486895284.0,0,0,1,1,1,103.0,15.0,2.0,1,1,2017-10-14,0.0,6.0
3,303949519.0,0,0,1,1,1,10.0,1.0,1.0,1,1,2015-04-15,30.0,4.0
4,301787904.0,0,0,1,1,1,9.0,1.0,0.0,0,0,NaT,NaN,NaN


In [912]:
year_indivs.shape

(4877220, 14)

## For those who purchased tires, what is their total sales for all years?

In [913]:
sales_tires_purch = dy_vhc_tire_purch.groupby('INDIV_ID').agg({'SALES':'sum'}).reset_index()

sales_tires_purch.columns = ['indiv', 'sales_tires_purch'] 

year_indivs = year_indivs.merge(sales_tires_purch, left_on = 'indiv', right_on = 'indiv', how = 'left')

year_indivs.head()

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans,number_visits,number_visits_tire_purchase,vhc_tire_purchase,vhc_early_tire_purchase,last_tire_purchase,diff_from_last_tire_purch,units_tires_purch,sales_tires_purch
0,295141827.0,0,0,1,1,1,13.0,2.0,0.0,0,0,NaT,NaN,NaN,NaN
1,303947384.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN
2,486895284.0,0,0,1,1,1,103.0,15.0,2.0,1,1,2017-10-14,0.0,6.0,475.22
3,303949519.0,0,0,1,1,1,10.0,1.0,1.0,1,1,2015-04-15,30.0,4.0,343.96
4,301787904.0,0,0,1,1,1,9.0,1.0,0.0,0,0,NaT,NaN,NaN,NaN


In [914]:
year_indivs.shape

(4877220, 15)

In [915]:
# year_indivs.to_csv(path + 'curated_data/part1_full_data.csv'))

## Feature: top customers regarding overall visits  
I assume visits occur on separate dates

In [916]:
number_trans_general = dy.groupby('INDIV_ID').agg({'DATE':'nunique'}).reset_index()

number_trans_general.columns = ['indiv', 'number_trans_general'] 

year_indivs = year_indivs.merge(number_trans_general, left_on = 'indiv', right_on = 'indiv', how = 'left')

year_indivs.head()

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans,number_visits,number_visits_tire_purchase,vhc_tire_purchase,vhc_early_tire_purchase,last_tire_purchase,diff_from_last_tire_purch,units_tires_purch,sales_tires_purch,number_trans_general
0,295141827.0,0,0,1,1,1,13.0,2.0,0.0,0,0,NaT,NaN,NaN,NaN,2
1,303947384.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,1
2,486895284.0,0,0,1,1,1,103.0,15.0,2.0,1,1,2017-10-14,0.0,6.0,475.22,17
3,303949519.0,0,0,1,1,1,10.0,1.0,1.0,1,1,2015-04-15,30.0,4.0,343.96,1
4,301787904.0,0,0,1,1,1,9.0,1.0,0.0,0,0,NaT,NaN,NaN,NaN,1


In [917]:
year_indivs.shape

(4877220, 16)

In [918]:
# top_cust_overall_list = dy.INDIV_ID.value_counts().nlargest(int(len(dy)/10))

In [919]:
# year_indivs['top_cust_overall'] = np.where(year_indivs['indiv'].isin(top_cust_overall_list), 1, 0)

In [920]:
## Feature: top customers regarding tires' purchase 
# top_cust_tire_purch_array = dy_vhc_tire_purch.INDIV_ID.value_counts().nlargest(int(len(dy_vhc_tire_purch)/10)).index
# year_indivs['top_cust_tire_purch'] = np.where(year_indivs['indiv'].isin(top_cust_tire_purch_array), 1, 0)
# year_indivs['top_cust_tire_purch'].value_counts()
# year_indivs.head(10)

## Feature: Top makes based on number of times a tire is purchased 

In [921]:
top_make_tire_purch_array = dy_vhc_tire_purch.MAKE.value_counts().nlargest(int(len(dy_vhc_tire_purch)/20)).index.tolist()[0:10]

In [922]:
top_make_tire_purch_array

['TOYOTA',
 'FORD',
 'CHEVROLET',
 'HONDA',
 'NISSAN',
 'DODGE',
 'HYUNDAI',
 'KIA',
 'CHRYSLER',
 'MAZDA']

In [923]:
dy['MAKE'] = dy['MAKE'].astype('str')

In [924]:
indiv_make_tire_purch = dy[dy['MAKE'].isin(top_make_tire_purch_array)]['INDIV_ID']

In [925]:
year_indivs['indiv_make_tire_pch'] = np.where(year_indivs.indiv.isin(indiv_make_tire_purch), 1, 0)

In [926]:
year_indivs['indiv_make_tire_pch'].value_counts()

1    3744592
0    1132628
Name: indiv_make_tire_pch, dtype: int64

In [927]:
year_indivs.head(10)

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans,number_visits,number_visits_tire_purchase,vhc_tire_purchase,vhc_early_tire_purchase,last_tire_purchase,diff_from_last_tire_purch,units_tires_purch,sales_tires_purch,number_trans_general,indiv_make_tire_pch
0,295141827.0,0,0,1,1,1,13.0,2.0,0.0,0,0,NaT,NaN,NaN,NaN,2,1
1,303947384.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,1,1
2,486895284.0,0,0,1,1,1,103.0,15.0,2.0,1,1,2017-10-14,0.0,6.0,475.22,17,1
3,303949519.0,0,0,1,1,1,10.0,1.0,1.0,1,1,2015-04-15,30.0,4.0,343.96,1,1
4,301787904.0,0,0,1,1,1,9.0,1.0,0.0,0,0,NaT,NaN,NaN,NaN,1,1
5,536567267.0,0,0,1,1,1,8.0,2.0,0.0,0,0,NaT,NaN,NaN,NaN,2,1
6,254367352.0,0,0,1,1,1,10.0,2.0,0.0,0,0,NaT,NaN,NaN,NaN,2,1
7,259258499.0,0,0,1,1,1,23.0,3.0,0.0,0,0,NaT,NaN,NaN,NaN,3,1
8,300257974.0,0,0,1,1,1,12.0,1.0,1.0,1,1,2015-04-22,30.0,4.0,261.00,1,1
9,304508907.0,0,0,1,1,1,6.0,1.0,1.0,1,1,2015-04-25,30.0,4.0,416.96,1,1


## Proportion of MAKE among all other MAKES

In [928]:
prop_make = dy.MAKE.value_counts(normalize = True).reset_index()

prop_make.columns = ['MAKE', 'make_prop']

In [929]:
prop_make.head()

,MAKE,make_prop
0,TOYOTA,0.132760
1,FORD,0.118321
2,CHEVROLET,0.112215
3,HONDA,0.103269
4,NISSAN,0.076727


I will assume here that each individual has a single car, which is the first car

In [930]:
dy_make = dy[['INDIV_ID', 'MAKE']]  #extract the columns ID and MAKE from original dataset - this can be used for the join

In [931]:
dy_make = dy_make.merge(prop_make, left_on = 'MAKE', right_on = 'MAKE', how = 'left')


In [932]:
dy_make.head()

,INDIV_ID,MAKE,make_prop
0,295141827.0,TOYOTA,0.13276
1,295141827.0,TOYOTA,0.13276
2,295141827.0,TOYOTA,0.13276
3,295141827.0,TOYOTA,0.13276
4,295141827.0,TOYOTA,0.13276


In [933]:
dy_make = dy_make.drop('MAKE', axis = 1)
dy_make.columns = ['indiv', 'make_prop']

In [934]:
dy_make.shape

(109858983, 2)

In [935]:
dy_make = dy_make.drop_duplicates(subset = ['indiv'])

dy_make.shape

(4877220, 2)

In [936]:
year_indivs = year_indivs.merge(dy_make, left_on = 'indiv', right_on = 'indiv', how = 'left')

In [937]:
year_indivs.head()

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans,number_visits,number_visits_tire_purchase,vhc_tire_purchase,vhc_early_tire_purchase,last_tire_purchase,diff_from_last_tire_purch,units_tires_purch,sales_tires_purch,number_trans_general,indiv_make_tire_pch,make_prop
0,295141827.0,0,0,1,1,1,13.0,2.0,0.0,0,0,NaT,NaN,NaN,NaN,2,1,0.132760
1,303947384.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,1,1,0.132760
2,486895284.0,0,0,1,1,1,103.0,15.0,2.0,1,1,2017-10-14,0.0,6.0,475.22,17,1,0.051176
3,303949519.0,0,0,1,1,1,10.0,1.0,1.0,1,1,2015-04-15,30.0,4.0,343.96,1,1,0.051176
4,301787904.0,0,0,1,1,1,9.0,1.0,0.0,0,0,NaT,NaN,NaN,NaN,1,1,0.051176


In [938]:
year_indivs.shape

(4877220, 18)

## Top MAKE based on proportion of tires purchased in comparision to total transactions

I Have to make sure that MAKE that were presented only once are ommitted

In [939]:
dy['tire_purchase'] = np.where((dy['PROD_GROUP_DESC'] == "Tires") & (dy['SEGMENT_DESC'] != "Others"), 1, 0)

df2 = dy.groupby(['MAKE', 'tire_purchase']).agg({'tire_purchase': 'count'})

df2.columns = ['total'] # I need to rename the column to handle multiindex caused by groupby 

df2 = df2.query('total > 100')

df3 = df2.groupby(level=0).apply(lambda x:100 * x / float(x.sum()))

top_make_tires_purch_prop_array = df3.reset_index().query('tire_purchase == 1').sort_values(by = 'total', ascending = False).nlargest(int(len(df3)/10),'total')['MAKE'].tolist()[0:1000]
                                                                                                                                      
indiv_make_tire_purch_prop = dy[dy['MAKE'].isin(top_make_tires_purch_prop_array)]['INDIV_ID']

year_indivs['make_prop_tire'] = np.where(year_indivs.indiv.isin(indiv_make_tire_purch_prop), 1, 0)

year_indivs['make_prop_tire'].value_counts()

0    4488433
1     388787
Name: make_prop_tire, dtype: int64

## Proportion of MODEL among all other MODELS

In [940]:
prop_MODEL = dy.MODEL.value_counts(normalize = True).reset_index()

prop_MODEL.columns = ['MODEL', 'MODEL_prop']

In [941]:
prop_MODEL.head()

,MODEL,MODEL_prop
0,CAMRY LE,0.020321
1,ACCORD EX,0.016936
2,ALTIMA S,0.011841
3,CIVIC EX,0.010650
4,COROLLA LE,0.010292


I will assume here that each individual has a single car, which is the first car

In [942]:
dy_MODEL = dy[['INDIV_ID', 'MODEL']]  #extract the columns ID and MODEL from original dataset - this can be used for the join

In [943]:
dy_MODEL = dy_MODEL.merge(prop_MODEL, left_on = 'MODEL', right_on = 'MODEL', how = 'left')


In [944]:
dy_MODEL.head()

,INDIV_ID,MODEL,MODEL_prop
0,295141827.0,COROLLA LE,0.010292
1,295141827.0,COROLLA LE,0.010292
2,295141827.0,COROLLA LE,0.010292
3,295141827.0,COROLLA LE,0.010292
4,295141827.0,COROLLA LE,0.010292


In [945]:
dy_MODEL = dy_MODEL.drop('MODEL', axis = 1)
dy_MODEL.columns = ['indiv', 'MODEL_prop']

In [946]:
dy_MODEL = dy_MODEL.drop_duplicates(subset = ['indiv'])

In [947]:
year_indivs = year_indivs.merge(dy_MODEL, left_on = 'indiv', right_on = 'indiv', how = 'left')

In [948]:
year_indivs.head()

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans,number_visits,number_visits_tire_purchase,vhc_tire_purchase,vhc_early_tire_purchase,last_tire_purchase,diff_from_last_tire_purch,units_tires_purch,sales_tires_purch,number_trans_general,indiv_make_tire_pch,make_prop,make_prop_tire,MODEL_prop
0,295141827.0,0,0,1,1,1,13.0,2.0,0.0,0,0,NaT,NaN,NaN,NaN,2,1,0.132760,0,0.010292
1,303947384.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,1,1,0.132760,0,0.004369
2,486895284.0,0,0,1,1,1,103.0,15.0,2.0,1,1,2017-10-14,0.0,6.0,475.22,17,1,0.051176,0,0.000058
3,303949519.0,0,0,1,1,1,10.0,1.0,1.0,1,1,2015-04-15,30.0,4.0,343.96,1,1,0.051176,0,0.000300
4,301787904.0,0,0,1,1,1,9.0,1.0,0.0,0,0,NaT,NaN,NaN,NaN,1,1,0.051176,0,0.001637


In [949]:
year_indivs.shape

(4877220, 20)

## Top MODEL based on proportion of tires purchased in comparision to total transactions

In [950]:
df2 = dy.groupby(['MODEL', 'tire_purchase']).agg({'tire_purchase': 'count'})

df2.columns = ['total'] # I need to rename the column to handle multiindex caused by groupby 

df2 = df2.query('total > 10')

df3 = df2.groupby(level=0).apply(lambda x:100 * x / float(x.sum()))

dy['MODEL'] = dy['MODEL'].astype('str')

top_MODEL_tires_purch_prop_array = df3.reset_index().query('tire_purchase == 1').sort_values(by = 'total', ascending = False).nlargest(int(len(df3)/10),'total')['MODEL'].tolist()[0:1000]

indiv_MODEL_tire_purch_prop = dy[dy['MODEL'].isin(top_MODEL_tires_purch_prop_array)]['INDIV_ID']
                                                                                                                                       
year_indivs['MODEL_prop_tire'] = np.where(year_indivs.indiv.isin(indiv_MODEL_tire_purch_prop), 1, 0)
                                                                                                                                       
year_indivs['MODEL_prop_tire'].value_counts()                                                                                                                                    

0    4729294
1     147926
Name: MODEL_prop_tire, dtype: int64

## Proportion of SUB_MODEL among all other SUB_MODELS

In [951]:
prop_SUB_MODEL = dy.SUB_MODEL.value_counts(normalize = True).reset_index()

prop_SUB_MODEL.columns = ['SUB_MODEL', 'SUB_MODEL_prop']

In [952]:
prop_SUB_MODEL = prop_SUB_MODEL.dropna(subset = ['SUB_MODEL'])

In [953]:
prop_SUB_MODEL.head()

,SUB_MODEL,SUB_MODEL_prop
0,Base,0.135851
1,EX,0.067239
2,SE,0.056124
3,Limited,0.043203
4,LX,0.039796


I will assume here that each individual has a single car, which is the first car

In [954]:
dy_SUB_MODEL = dy[['INDIV_ID', 'SUB_MODEL']]  #extract the columns ID and SUB_MODEL from original dataset - this can be used for the join

In [955]:
dy_SUB_MODEL = dy_SUB_MODEL.dropna(subset = ['SUB_MODEL'])

In [956]:
dy_SUB_MODEL = dy_SUB_MODEL.drop_duplicates(subset = ['SUB_MODEL'])

In [957]:
dy_SUB_MODEL = dy_SUB_MODEL.merge(prop_SUB_MODEL, left_on = 'SUB_MODEL', right_on = 'SUB_MODEL', how = 'left')

In [958]:
dy_SUB_MODEL.head()

,INDIV_ID,SUB_MODEL,SUB_MODEL_prop
0,303947384.0,S,0.039052
1,486895284.0,C/V,0.000085
2,303949519.0,R/T,0.002137
3,254367352.0,S 2.5 Sedan,0.001533
4,259258499.0,LS,0.035741


In [959]:
dy_SUB_MODEL = dy_SUB_MODEL.drop('SUB_MODEL', axis = 1)

dy_SUB_MODEL.columns = ['indiv', 'SUB_MODEL_prop']

In [960]:
dy_SUB_MODEL = dy_SUB_MODEL.dropna(subset = ['indiv'])
dy_SUB_MODEL.drop_duplicates(subset = ['indiv'], inplace = True)

In [961]:
year_indivs = year_indivs.merge(dy_SUB_MODEL, left_on = 'indiv', right_on = 'indiv', how = 'left')

In [962]:
year_indivs.shape

(4877220, 22)

## Top SUB_MODEL based on proportion of tires purchased in comparision to total transactions

In [963]:
df2 = dy.groupby(['SUB_MODEL', 'tire_purchase']).agg({'tire_purchase': 'count'})

df2.columns = ['total'] # I need to rename the column to handle multiindex caused by groupby 

df2 = df2.query('total > 10')

df3 = df2.groupby(level=0).apply(lambda x:100 * x / float(x.sum()))

dy['SUB_MODEL'] = dy['SUB_MODEL'].astype('str')

top_SUB_MODEL_tires_purch_prop_array = df3.reset_index().query('tire_purchase == 1').sort_values(by = 'total', ascending = False).nlargest(int(len(df3)/10), 'total')['SUB_MODEL'].tolist()[0:1000]            

indiv_SUB_MODEL_tire_purch_prop = dy[dy['SUB_MODEL'].isin(top_SUB_MODEL_tires_purch_prop_array)]['INDIV_ID']
                                                                                                                                               
year_indivs['SUB_MODEL_prop_tire'] = np.where(year_indivs.indiv.isin(indiv_SUB_MODEL_tire_purch_prop), 1, 0)
                                                                                                                                               
year_indivs['SUB_MODEL_prop_tire'].value_counts()

0    4700759
1     176461
Name: SUB_MODEL_prop_tire, dtype: int64

In [964]:
year_indivs.head(10)

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans,number_visits,number_visits_tire_purchase,vhc_tire_purchase,vhc_early_tire_purchase,last_tire_purchase,diff_from_last_tire_purch,units_tires_purch,sales_tires_purch,number_trans_general,indiv_make_tire_pch,make_prop,make_prop_tire,MODEL_prop,MODEL_prop_tire,SUB_MODEL_prop,SUB_MODEL_prop_tire
0,295141827.0,0,0,1,1,1,13.0,2.0,0.0,0,0,NaT,NaN,NaN,NaN,2,1,0.132760,0,0.010292,0,NaN,0
1,303947384.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,1,1,0.132760,0,0.004369,0,0.039052,0
2,486895284.0,0,0,1,1,1,103.0,15.0,2.0,1,1,2017-10-14,0.0,6.0,475.22,17,1,0.051176,0,0.000058,0,0.000085,0
3,303949519.0,0,0,1,1,1,10.0,1.0,1.0,1,1,2015-04-15,30.0,4.0,343.96,1,1,0.051176,0,0.000300,0,0.002137,0
4,301787904.0,0,0,1,1,1,9.0,1.0,0.0,0,0,NaT,NaN,NaN,NaN,1,1,0.051176,0,0.001637,0,NaN,0
5,536567267.0,0,0,1,1,1,8.0,2.0,0.0,0,0,NaT,NaN,NaN,NaN,2,1,0.132760,0,0.003018,0,NaN,0
6,254367352.0,0,0,1,1,1,10.0,2.0,0.0,0,0,NaT,NaN,NaN,NaN,2,1,0.076727,0,0.011841,0,0.001533,0
7,259258499.0,0,0,1,1,1,23.0,3.0,0.0,0,0,NaT,NaN,NaN,NaN,3,1,0.112215,0,0.002735,0,0.035741,0
8,300257974.0,0,0,1,1,1,12.0,1.0,1.0,1,1,2015-04-22,30.0,4.0,261.00,1,1,0.118321,0,0.006132,0,0.056124,0
9,304508907.0,0,0,1,1,1,6.0,1.0,1.0,1,1,2015-04-25,30.0,4.0,416.96,1,1,0.118321,0,0.004904,0,0.011484,0


## SEGMENT Description

In [965]:
# func that returns a dummified DataFrame of significant dummies in a given column
def dum_sign(dummy_col, threshold=0.1, columns = None):

    # removes the bind
    dummy_col = dummy_col.copy()

    # what is the ratio of a dummy in whole column
    count = pd.value_counts(dummy_col.columns) / len(dummy_col.columns)

    # cond whether the ratios is higher than the threshold
    mask = dummy_col.isin(count[count > threshold].index)

    # replace the ones which ratio is lower than the threshold by a special name
    dummy_col[columns][~mask] = "others"

    return pd.get_dummies(dummy_col, prefix=dummy_col.columns.name, drop_first = True, columns = columns)

In [966]:
# dataset after filtering for tire purchases
dy_tire_segment = dy_vhc_tire_purch[['INDIV_ID','SEGMENT_DESC']]

dy_tire_segment = dum_sign(dy_tire_segment, columns = ['SEGMENT_DESC'])

dy_tire_segment = dy_tire_segment.rename(columns = {'INDIV_ID' : 'indiv'})

dy_tire_segment = dy_tire_segment.drop_duplicates(subset = ['indiv'])

year_indivs = year_indivs.merge(dy_tire_segment, left_on = 'indiv', right_on = 'indiv', how = 'left')

/data/p_dsi/envs/neuro501/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/data/p_dsi/envs/neuro501/lib/python3.7/site-packages/pandas/core/frame.py:3718: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


In [967]:
year_indivs.head(200)

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans,number_visits,number_visits_tire_purchase,vhc_tire_purchase,vhc_early_tire_purchase,last_tire_purchase,diff_from_last_tire_purch,units_tires_purch,sales_tires_purch,number_trans_general,indiv_make_tire_pch,make_prop,make_prop_tire,MODEL_prop,MODEL_prop_tire,SUB_MODEL_prop,SUB_MODEL_prop_tire,SEGMENT_DESC_Commercial Tires,SEGMENT_DESC_Commercial/Recreational,SEGMENT_DESC_Industrial Tires,SEGMENT_DESC_Medium Truck Tires,SEGMENT_DESC_Miscellaneous Tire Related Items,SEGMENT_DESC_Other Passenger Tires,SEGMENT_DESC_P Metric Light Truck Tires,SEGMENT_DESC_P-Metric CUV/SUV,SEGMENT_DESC_Performance Tires,SEGMENT_DESC_Recreational Tires,SEGMENT_DESC_Road Hazard,SEGMENT_DESC_Snow Tires-LT,SEGMENT_DESC_Snow Tires-PS,SEGMENT_DESC_Touring,SEGMENT_DESC_Tubes
0,295141827.0,0,0,1,1,1,13.0,2.0,0.0,0,0,NaT,NaN,NaN,NaN,2,1,0.132760,0,0.010292,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,303947384.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,1,1,0.132760,0,0.004369,0,3.905232e-02,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,486895284.0,0,0,1,1,1,103.0,15.0,2.0,1,1,2017-10-14,0.0,6.0,475.22,17,1,0.051176,0,0.000058,0,8.473666e-05,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,303949519.0,0,0,1,1,1,10.0,1.0,1.0,1,1,2015-04-15,30.0,4.0,343.96,1,1,0.051176,0,0.000300,0,2.136587e-03,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,301787904.0,0,0,1,1,1,9.0,1.0,0.0,0,0,NaT,NaN,NaN,NaN,1,1,0.051176,0,0.001637,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,536567267.0,0,0,1,1,1,8.0,2.0,0.0,0,0,NaT,NaN,NaN,NaN,2,1,0.132760,0,0.003018,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,254367352.0,0,0,1,1,1,10.0,2.0,0.0,0,0,NaT,NaN,NaN,NaN,2,1,0.076727,0,0.011841,0,1.533196e-03,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,259258499.0,0,0,1,1,1,23.0,3.0,0.0,0,0,NaT,NaN,NaN,NaN,3,1,0.112215,0,0.002735,0,3.574143e-02,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,300257974.0,0,0,1,1,1,12.0,1.0,1.0,1,1,2015-04-22,30.0,4.0,261.00,1,1,0.118321,0,0.006132,0,5.612351e-02,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,304508907.0,0,0,1,1,1,6.0,1.0,1.0,1,1,2015-04-25,30.0,4.0,416.96,1,1,0.118321,0,0.004904,0,1.148362e-02,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [968]:
year_indivs.shape

(4877220, 38)

## Individuals in top 5 segments 

In [969]:
segment_tire_sales = dy[dy['PROD_GROUP_CODE']==5].groupby(['SEGMENT_DESC'])['SALES'].count().to_frame()
segment_tire_sales.rename(columns = {'SALES': 'NUMBER_OF_TIRE_SALES'},inplace = True)
top_5_segment_tire_sales = segment_tire_sales.sort_values('NUMBER_OF_TIRE_SALES', ascending = False)[:5]
top_5_segment_tire_sales


,NUMBER_OF_TIRE_SALES
SEGMENT_DESC,
Road Hazard,2859614
Touring,1607301
Performance Tires,1382848
P Metric Light Truck Tires,594079
P-Metric CUV/SUV,563451


In [970]:
top_5_segment_tire_sales.reset_index(inplace = True)
top_5_segment_tire_sales = top_5_segment_tire_sales['SEGMENT_DESC']
indiv_top_5_segment_tire_sales = dy[dy['SEGMENT_DESC'].isin(top_5_segment_tire_sales)]['INDIV_ID']
year_indivs['top_5_segment'] = np.where(year_indivs.indiv.isin(indiv_top_5_segment_tire_sales), 1, 0)
year_indivs['top_5_segment'].value_counts()

0    2537039
1    2340181
Name: top_5_segment, dtype: int64

## CLASS Description

In [971]:
# dataset after filtering for tire purchases      # WILL BE used as a backup plan 
dy_tire_CLASS = dy_vhc_tire_purch[['INDIV_ID','CLASS_DESC']]

dy_tire_CLASS = dum_sign(dy_tire_CLASS, columns = ['CLASS_DESC'])

dy_tire_CLASS = dy_tire_CLASS.rename(columns = {'INDIV_ID' : 'indiv'})

dy_tire_CLASS = dy_tire_CLASS.drop_duplicates(subset = ['indiv'])

year_indivs = year_indivs.merge(dy_tire_CLASS, left_on = 'indiv', right_on = 'indiv', how = 'left')

/data/p_dsi/envs/neuro501/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/data/p_dsi/envs/neuro501/lib/python3.7/site-packages/pandas/core/frame.py:3718: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


## Tire service

In [972]:
dy_tire_service = dy.query('CATEGORY_DESC == "Tire Services" & CLASS_DESC != "Discount Tire Services"')

In [973]:
# dy_tire_service.CLASS_DESC.value_counts()

In [974]:
dy_service = dy_tire_service[['INDIV_ID','CLASS_DESC']]

dy_service = dum_sign(dy_service, columns = ['CLASS_DESC'])

dy_service = dy_service.rename(columns = {'INDIV_ID' : 'indiv'})

dy_service.drop_duplicates(subset = ['indiv'], inplace = True)

year_indivs = year_indivs.merge(dy_service, left_on = 'indiv', right_on = 'indiv', how = 'left')

/data/p_dsi/envs/neuro501/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [975]:
year_indivs.head()

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans,number_visits,number_visits_tire_purchase,vhc_tire_purchase,vhc_early_tire_purchase,last_tire_purchase,diff_from_last_tire_purch,units_tires_purch,sales_tires_purch,number_trans_general,indiv_make_tire_pch,make_prop,make_prop_tire,MODEL_prop,MODEL_prop_tire,SUB_MODEL_prop,SUB_MODEL_prop_tire,SEGMENT_DESC_Commercial Tires,SEGMENT_DESC_Commercial/Recreational,SEGMENT_DESC_Industrial Tires,SEGMENT_DESC_Medium Truck Tires,SEGMENT_DESC_Miscellaneous Tire Related Items,SEGMENT_DESC_Other Passenger Tires,SEGMENT_DESC_P Metric Light Truck Tires,SEGMENT_DESC_P-Metric CUV/SUV,SEGMENT_DESC_Performance Tires,SEGMENT_DESC_Recreational Tires,SEGMENT_DESC_Road Hazard,SEGMENT_DESC_Snow Tires-LT,SEGMENT_DESC_Snow Tires-PS,SEGMENT_DESC_Touring,SEGMENT_DESC_Tubes,top_5_segment,CLASS_DESC_All-Terrain,CLASS_DESC_CUV/SUV All-Terrain,CLASS_DESC_CUV/SUV Highway All-Season,CLASS_DESC_CUV/SUV Sport Performance Summer,CLASS_DESC_Car & Minivan All-Season,CLASS_DESC_Economy Tires,CLASS_DESC_Garden Tires,CLASS_DESC_Highway All-Season,CLASS_DESC_Highway Tires,CLASS_DESC_Implement Tires/Industrial,CLASS_DESC_Industrial Tires,CLASS_DESC_Mass Market Tires,CLASS_DESC_Max Traction Tires,CLASS_DESC_Maximum Traction,CLASS_DESC_Misc/Discounts/Non-Inventoreable,CLASS_DESC_Other,CLASS_DESC_Other Passenger Tires,CLASS_DESC_Performance All-Season,CLASS_DESC_Radial Tires,CLASS_DESC_Rear Ag Tires,CLASS_DESC_Road Hazard,CLASS_DESC_S/T Performance Tires,CLASS_DESC_Snow Tires,CLASS_DESC_Snow Tires - All Terrain LT,CLASS_DESC_Snow Tires - Commercial LT,CLASS_DESC_Touring H/V/Z Tires,CLASS_DESC_Tubes,CLASS_DESC_Ultra High Performance All Season,CLASS_DESC_Ultra High Performance Summer,CLASS_DESC_Farm/Truck Services,CLASS_DESC_Misc. Tire Service,CLASS_DESC_Tire Mounting,CLASS_DESC_Tire Pressure Monitoring Systems,CLASS_DESC_Tire Rotation,CLASS_DESC_Valves
0,295141827.0,0,0,1,1,1,13.0,2.0,0.0,0,0,NaT,NaN,NaN,NaN,2,1,0.132760,0,0.010292,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0
1,303947384.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,1,1,0.132760,0,0.004369,0,0.039052,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,486895284.0,0,0,1,1,1,103.0,15.0,2.0,1,1,2017-10-14,0.0,6.0,475.22,17,1,0.051176,0,0.000058,0,0.000085,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,303949519.0,0,0,1,1,1,10.0,1.0,1.0,1,1,2015-04-15,30.0,4.0,343.96,1,1,0.051176,0,0.000300,0,0.002137,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,301787904.0,0,0,1,1,1,9.0,1.0,0.0,0,0,NaT,NaN,NaN,NaN,1,1,0.051176,0,0.001637,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0


In [976]:
year_indivs.shape

(4877220, 74)

In [977]:
# year_indivs.to_csv('individuals18.csv')

In [978]:
dy = dy.replace('NONE', np.nan) # removing NONE from the 3 columns 

In [979]:
dy_tire_specs = dy[['INDIV_ID', 'VEHICLE_ID','CROSS_SECTION','ASPECT_RATIO', 'RIM_SIZE']]

In [980]:
dy_tire_specs.CROSS_SECTION = dy_tire_specs.CROSS_SECTION.astype('float')
dy_tire_specs.ASPECT_RATIO = dy_tire_specs.ASPECT_RATIO.astype('float')
dy_tire_specs.RIM_SIZE = dy_tire_specs.RIM_SIZE.astype('float')

/data/p_dsi/envs/neuro501/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [981]:
dy_tire_specs = dy_tire_specs.groupby('INDIV_ID').agg({'CROSS_SECTION' : 'min','ASPECT_RATIO': 'min'
                                                       , 'RIM_SIZE': 'min'})

In [982]:
dy_tire_specs = dy_tire_specs.reset_index()

In [983]:
dy_tire_specs = dy_tire_specs.rename(columns = {'INDIV_ID' : 'indiv'})

In [984]:
dy_service.drop_duplicates(subset = ['indiv'], inplace = True)


In [985]:
dy_tire_specs.head()

,indiv,CROSS_SECTION,ASPECT_RATIO,RIM_SIZE
0,251634506.0,205.0,55.0,16.0
1,251634507.0,235.0,45.0,17.0
2,251634508.0,225.0,60.0,16.0
3,251634512.0,NaN,NaN,NaN
4,251634532.0,NaN,NaN,NaN


In [986]:
year_indivs.head()

,indiv,label,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,number_trans,number_visits,number_visits_tire_purchase,vhc_tire_purchase,vhc_early_tire_purchase,last_tire_purchase,diff_from_last_tire_purch,units_tires_purch,sales_tires_purch,number_trans_general,indiv_make_tire_pch,make_prop,make_prop_tire,MODEL_prop,MODEL_prop_tire,SUB_MODEL_prop,SUB_MODEL_prop_tire,SEGMENT_DESC_Commercial Tires,SEGMENT_DESC_Commercial/Recreational,SEGMENT_DESC_Industrial Tires,SEGMENT_DESC_Medium Truck Tires,SEGMENT_DESC_Miscellaneous Tire Related Items,SEGMENT_DESC_Other Passenger Tires,SEGMENT_DESC_P Metric Light Truck Tires,SEGMENT_DESC_P-Metric CUV/SUV,SEGMENT_DESC_Performance Tires,SEGMENT_DESC_Recreational Tires,SEGMENT_DESC_Road Hazard,SEGMENT_DESC_Snow Tires-LT,SEGMENT_DESC_Snow Tires-PS,SEGMENT_DESC_Touring,SEGMENT_DESC_Tubes,top_5_segment,CLASS_DESC_All-Terrain,CLASS_DESC_CUV/SUV All-Terrain,CLASS_DESC_CUV/SUV Highway All-Season,CLASS_DESC_CUV/SUV Sport Performance Summer,CLASS_DESC_Car & Minivan All-Season,CLASS_DESC_Economy Tires,CLASS_DESC_Garden Tires,CLASS_DESC_Highway All-Season,CLASS_DESC_Highway Tires,CLASS_DESC_Implement Tires/Industrial,CLASS_DESC_Industrial Tires,CLASS_DESC_Mass Market Tires,CLASS_DESC_Max Traction Tires,CLASS_DESC_Maximum Traction,CLASS_DESC_Misc/Discounts/Non-Inventoreable,CLASS_DESC_Other,CLASS_DESC_Other Passenger Tires,CLASS_DESC_Performance All-Season,CLASS_DESC_Radial Tires,CLASS_DESC_Rear Ag Tires,CLASS_DESC_Road Hazard,CLASS_DESC_S/T Performance Tires,CLASS_DESC_Snow Tires,CLASS_DESC_Snow Tires - All Terrain LT,CLASS_DESC_Snow Tires - Commercial LT,CLASS_DESC_Touring H/V/Z Tires,CLASS_DESC_Tubes,CLASS_DESC_Ultra High Performance All Season,CLASS_DESC_Ultra High Performance Summer,CLASS_DESC_Farm/Truck Services,CLASS_DESC_Misc. Tire Service,CLASS_DESC_Tire Mounting,CLASS_DESC_Tire Pressure Monitoring Systems,CLASS_DESC_Tire Rotation,CLASS_DESC_Valves
0,295141827.0,0,0,1,1,1,13.0,2.0,0.0,0,0,NaT,NaN,NaN,NaN,2,1,0.132760,0,0.010292,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0
1,303947384.0,0,0,1,1,0,0.0,0.0,0.0,0,0,NaT,NaN,NaN,NaN,1,1,0.132760,0,0.004369,0,0.039052,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,486895284.0,0,0,1,1,1,103.0,15.0,2.0,1,1,2017-10-14,0.0,6.0,475.22,17,1,0.051176,0,0.000058,0,0.000085,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,303949519.0,0,0,1,1,1,10.0,1.0,1.0,1,1,2015-04-15,30.0,4.0,343.96,1,1,0.051176,0,0.000300,0,0.002137,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,301787904.0,0,0,1,1,1,9.0,1.0,0.0,0,0,NaT,NaN,NaN,NaN,1,1,0.051176,0,0.001637,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0


In [987]:
year_indivs = year_indivs.merge(dy_tire_specs, left_on = 'indiv', right_on = 'indiv', how = 'left')

In [988]:
year_indivs.shape

(4877220, 77)

In [989]:
del dy_tire_specs
gc.collect()

40

In [990]:
# year_indivs.to_csv(path + 'curated_data/year_indivs.csv')

## Feature: Acceptable percntage of miles within a state

The higher the percentage, the more suited the road for driving

In [991]:
dy_state_zip = dy[['INDIV_ID','STATE_CODE', 'ZIP_CODE']]

dy_state_zip.columns = ['indiv','STATE_CODE', 'ZIP_CODE']

In [992]:
dy_state_zip.drop_duplicates(subset = 'indiv', inplace = True)

/data/p_dsi/envs/neuro501/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [993]:
year_indivs = year_indivs.merge(dy_state_zip, left_on = 'indiv', right_on = 'indiv', how = 'left')

In [994]:
accept_perc = pd.read_csv(path + 'supp_data/curated_supp_data/accept_miles.csv', index_col = 0)

In [995]:
accept_perc.head()

,Code,Percent Acceptable
3,AL,0.861377
4,AK,0.775208
5,AZ,0.803321
6,AR,0.912018
7,CA,0.550144


In [996]:
accept_perc.columns = ['STATE_CODE', 'accept_perc']

In [997]:
accept_perc.drop_duplicates(subset = 'STATE_CODE', inplace = True)

year_indivs = year_indivs.merge(accept_perc, left_on = 'STATE_CODE', right_on = 'STATE_CODE', how = 'left')

In [998]:
del dy_state_zip
del accept_perc
del dy 

gc.collect()

100

## Feature: weather per state

In [999]:
weather_state = pd.read_csv(path + 'supp_data/curated_supp_data/weather_state.csv', index_col = 0)

In [1000]:
weather_state.head()

,state,year,month,avg_temp,avg_max_temp,avg_prcp,avg_snow_depth
0,AK,2015,1,17.971178,72.990438,2.379781,475.543727
1,AK,2015,2,20.759009,49.222243,3.012146,345.913970
2,AK,2015,3,22.334774,71.193844,2.818260,492.077742
3,AK,2015,4,31.860780,42.486386,3.655609,503.802364
4,AK,2015,5,46.103190,64.287331,2.700007,734.360541


I will not use time series. Thus, I will use the average across all years

In [1001]:
weather_state = weather_state.groupby('state').agg({'avg_temp' : 'mean', 'avg_max_temp' : 'max',
                                  'avg_prcp': 'mean', 'avg_snow_depth': 'mean'}).reset_index()

In [1002]:
weather_state = weather_state.rename(columns = {'state' : 'STATE_CODE'})

In [1003]:
year_indivs = year_indivs.merge(weather_state, left_on = 'STATE_CODE', right_on = 'STATE_CODE', how = 'left')

In [1004]:
del weather_state

gc.collect()

120

In [1005]:

# accept_perc.drop_duplicates(subset = 'STATE_CODE', inplace = True)

## Feature: average miles covered

In [1006]:
miles_covered = pd.read_csv(path + 'supp_data/curated_supp_data/zip_county.csv', index_col = 0)

In [1007]:
miles_covered.head()

,ZIP,est_vmiles
0,19904,45.302813
1,19980,45.302813
2,19952,45.302813
3,19962,45.302813
4,19901,45.302813


In [1008]:
miles_covered = miles_covered.rename(columns = {'ZIP' : 'ZIP_CODE'})

miles_covered.drop_duplicates(subset = ['ZIP_CODE'], inplace = True)

In [1009]:
year_indivs = year_indivs.merge(miles_covered, left_on = 'ZIP_CODE', right_on = 'ZIP_CODE', how = 'left')

In [1010]:
year_indivs.shape

(4877220, 85)

In [1011]:
del miles_covered

gc.collect()

40

In [1012]:
# ddf = dd.from_pandas(year_indivs, npartitions=10) ## parallelizing using dask

In [1013]:
ddf.to_csv(path + 'curated_data/sample_data-*.csv', index=False)

['/gpfs52/data/p_dsi/teams2022/team_4/curated_data/sample_data-0.csv',
 '/gpfs52/data/p_dsi/teams2022/team_4/curated_data/sample_data-1.csv',
 '/gpfs52/data/p_dsi/teams2022/team_4/curated_data/sample_data-2.csv',
 '/gpfs52/data/p_dsi/teams2022/team_4/curated_data/sample_data-3.csv',
 '/gpfs52/data/p_dsi/teams2022/team_4/curated_data/sample_data-4.csv',
 '/gpfs52/data/p_dsi/teams2022/team_4/curated_data/sample_data-5.csv',
 '/gpfs52/data/p_dsi/teams2022/team_4/curated_data/sample_data-6.csv',
 '/gpfs52/data/p_dsi/teams2022/team_4/curated_data/sample_data-7.csv',
 '/gpfs52/data/p_dsi/teams2022/team_4/curated_data/sample_data-8.csv',
 '/gpfs52/data/p_dsi/teams2022/team_4/curated_data/sample_data-9.csv']

In [1014]:
# year_indivs.to_csv(path + 'curated_data/sample_data-*.csv'
#          , sep='|'
#          , header=True
#          , index=False
#          , chunksize=100000
#          , compression='gzip'
#          , encoding='utf-8')

In [1015]:
year_indivs.to_csv(path + 'curated_data/trial.csv')